In [1]:
import pandas as pd

In [3]:
# 工作流程：
##１　特征提取
## 2  提取叶子节点特征(gbdt抽特征)
##３　整体丢进ffm
## 4  其他处理

In [4]:
# 1 数据读入和预处理

In [5]:

import time
import pandas as pd
# 今晚先看看别人怎么搞事情滴
pd.options.display.max_columns = None
import lightgbm as lgb
# 注意到sklearn 提供log_loss 计算方法
from sklearn.metrics import log_loss

In [6]:
# 时间转换和分割代码
def timestamp_datetime(value):
    format = '%Y-%m-%d %H:%M:%S'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return dt

data = pd.read_csv('round1_ijcai_18_train_20180301.txt', sep=' ')

In [7]:
data.drop_duplicates(inplace=True)
# 处理时间戳
data['time'] = data.context_timestamp.apply(timestamp_datetime)

In [9]:
data.head(2)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_id,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,time
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,4505772604969228686,1,1003,2005,3003,282924576738839389,1537236544,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,2018-09-18 10:09:04
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,2692638157208937547,0,1002,2005,3006,4007979028023783431,1537243232,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,2018-09-18 12:00:32


# 先回顾过去的一般流程：
## step1: count.py
   - 统计在每一个变量上的过去的值0/1/总数的数量
   - 统计pos/total的数量
   
## step2:pre-a.py
   - 对dense类变量和sparse类变量分别进行gbdt分类处理，生成gbdt所需要的数据格式
   - 生成gbdtDense，这里对于dense的处理很有趣
      - 允许缺失
      - 允许C2- 即表明C2未知
   - 生成gbdtSparse
   
## step3：
   - 执行gbdt
   - 生成gbdt的output

## step 4 ：
   - 准被ffm所需要的数据
   - 这里连续变量也被hashTicks了，不清楚这样做的含义

### 1 count 统计的内容
- Field,Value,Neg,Pos,Total,Ratio


### 2 生成的 gbdt.dense , gbdt.sparse 的内容
- dense:就是给出了连续属性
- sparse:
   - 给出了一个target_cat_feats 的list
   - 返回每一列具有target_cat_feats的哪几个
- 需要注意在train 和 test　上的loss 来调整树的参数

### 3 生成gbdt.out
- 返回每棵树给出的节点

### 4 生成ffm所需要的数据
- read_freqent_feats(10)
   - 这是一个阈值处理：
   - 找到总共出现次数大于10的变量
- 生成ffm文件：
   - 1 对于原始变量预处理：
       - 连续变量
           - 如果value不 为空：
               - 如果>2:
                   处理为 int(math.log(float(value))**2)
               - 如果不大2：
                   value = 'SP'+str(value)
               - 返回key = field + '-' + str(value) 如 ‘I1-SP1’，‘I3-2’
               - 注意这里对于连续变量离散化了
       - 离散变量
           - 直接正常进行返回‘C1-68fd1e64’
       - 进行阈值处理：
           - C1-68fd1e64 若出现次数不高，将其化为'C1less
   - 2 对于树变量的处理：
       - 1:147 表示第一颗树的叶子节点是哪个
   - 3 获得了一个变量大概如下
```   
[(1, 'I1-SP1'),
 (2, 'I2-SP1'),
 (3, 'I3-2'),
 (4, 'I4-SP0'),
 (5, 'I5-52'),
 (6, 'I6-1'),
 (7, 'I7-7'),
 (8, 'I8-SP2'),
 (9, 'I9-27'),
 (10, 'I10-SP1'),
 (11, 'I11-SP2'),
 (12, 'I12-'),
 (13, 'I13-SP2'),
 (14, 'C1-68fd1e64'),
 (15, 'C2-80e26c9b'),
 (16, 'C3less'),
 (17, 'C4less'),
 (18, 'C5-25c83c98'),
 (19, 'C6-7e0ccccf'),
 (20, 'C7less'),
 (21, 'C8-1f89b562'),
 (22, 'C9-a73ee510'),
 (23, 'C10less'),
 (24, 'C11less'),
 (25, 'C12less'),
 (26, 'C13less'),
 (27, 'C14-1adce6ef'),
 (28, 'C15less'),
 (29, 'C16less'),
 (30, 'C17-e5ba7672'),
 (31, 'C18-f54016b9'),
 (32, 'C19-21ddcdc9'),
 (33, 'C20-b1252a9d'),
 (34, 'C21less'),
 (35, 'C22-'),
 (36, 'C23-3a171ecb'),
 (37, 'C24less'),
 (38, 'C25-e8b83407'),
 (39, 'C26less'),
 (40, '1:147'),
 (41, '2:232'),
 (42, '3:226'),
 (43, '4:142'),
 (44, '5:227'),
 (45, '6:228'),
 (46, '7:194'),
 (47, '8:202'),
 (48, '9:196'),
 (49, '10:248'),
 (50, '11:193'),
 (51, '12:147'),
 (52, '13:159'),
 (53, '14:248'),
 (54, '15:213'),
 (55, '16:135'),
 (56, '17:136'),
 (57, '18:150'),
 (58, '19:143'),
 (59, '20:132'),
 (60, '21:152'),
 (61, '22:129'),
 (62, '23:184'),
 (63, '24:216'),
 (64, '25:136'),
 (65, '26:224'),
 (66, '27:128'),
 (67, '28:236'),
 (68, '29:233'),
 (69, '30:247')]
 ```
    - 4 对变量的值进行hash化，使用hashTricks
```
['0:40189:1',
 '1:498397:1',
 '2:131438:1',
 '3:947702:1',
 '4:205745:1',
 '5:786172:1',
 '6:754008:1',
 '7:514500:1',
 '8:735727:1',
 '9:255381:1',
 '10:756430:1',
 '11:832677:1',
 '12:120252:1',
 '13:172672:1',
 '14:398230:1',
 '15:98079:1',
 '16:550602:1',
 '17:397270:1',
 '18:182671:1',
 '19:760878:1',
 '20:241196:1',
 '21:198788:1',
 '22:538959:1',
 '23:295561:1',
 '24:540660:1',
 '25:391696:1',
 '26:78061:1',
 '27:462176:1',
 '28:433710:1',
 '29:166818:1',
 '30:755327:1',
 '31:765122:1',
 '32:382381:1',
 '33:758475:1',
 '34:541960:1',
 '35:979212:1',
 '36:345058:1',
 '37:396665:1',
 '38:254077:1',
 '39:783960:1',
 '40:813207:1',
 '41:554970:1',
 '42:23531:1',
 '43:536736:1',
 '44:408760:1',
 '45:977232:1',
 '46:981376:1',
 '47:866651:1',
 '48:916980:1',
 '49:206496:1',
 '50:532334:1',
 '51:613587:1',
 '52:646650:1',
 '53:317482:1',
 '54:299876:1',
 '55:671775:1',
 '56:487158:1',
 '57:547606:1',
 '58:184337:1',
 '59:403504:1',
 '60:774722:1',
 '61:712399:1',
 '62:98186:1',
 '63:96827:1',
 '64:529173:1',
 '65:371541:1',
 '66:30824:1',
 '67:504054:1',
 '68:394338:1']
```

### 5 使用libFFM来进行预测（略）

### 借鉴上述思路，我们应该做的是
   - 统计和划归每个类别出现次数较少的情况
   - 对连续变量合理的桶化
   - 对于属性变量单独建树
   - 对连续，离散变量树
   - 抽取合理的特征建树
   - 提取尽量多的id类特征
   - 考察不平衡的处理方法